<a href="https://colab.research.google.com/github/Nuri0063/FilmYorumAnaliz_LLM/blob/main/LLMg%C3%BCncel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install transformers datasets torch scikit-learn -q

# GPU kontrolü
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Kullanılan cihaz: cuda


In [ ]:
from datasets import load_dataset

# IMDB veri setini yükle
print("Veri seti yükleniyor...")
dataset = load_dataset("imdb")  # IMDB film yorumları veri seti

# Eğitim ve değerlendirme veri setlerini küçült
print("Veri seti küçültülüyor...")
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(5000))  # Eğitim için ilk 5000 veri
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(1000))    # Değerlendirme için ilk 1000 veri


Veri seti yükleniyor...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Veri seti küçültülüyor...


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Tokenizer ve model yükleme
print("Tokenizer ve model yükleniyor...")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)  # Modeli GPU/CPU'ya taşı


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Tokenizer ve model yükleniyor...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# Tokenize etme fonksiyonu
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Eğitim ve değerlendirme veri setlerini tokenize et
print("Veri seti tokenize ediliyor...")
tokenized_train = small_train_dataset.map(tokenize_function, batched=True)
tokenized_eval = small_eval_dataset.map(tokenize_function, batched=True)

# Veri formatını ayarlayın
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval = tokenized_eval.rename_column("label", "labels")
tokenized_eval.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

print("Tokenizasyon işlemi tamamlandı.")


Veri seti tokenize ediliyor...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizasyon işlemi tamamlandı.


In [ ]:
from transformers import TrainingArguments

# Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./results",                 # Sonuçların kaydedileceği dizin
    evaluation_strategy="epoch",           # Her epoch sonunda değerlendirme yapılacak
    save_strategy="epoch",                 # Her epoch sonunda model kaydedilecek
    learning_rate=5e-5,                    # Öğrenme oranı
    per_device_train_batch_size=16,        # Eğitim için batch size
    per_device_eval_batch_size=32,         # Değerlendirme için batch size
    num_train_epochs=5,                    # Eğitim için epoch sayısı
    weight_decay=0.01,                     # Ağırlık azalma oranı
    logging_dir="./logs",                  # Log dosyaları
    logging_steps=50,                      # Loglama adımları
    save_total_limit=2,                    # En fazla 2 model saklanacak
    load_best_model_at_end=True            # En iyi modeli sakla
)

print("Eğitim ayarları belirlendi.")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Eğitim ayarları belirlendi.


In [ ]:
from sklearn.metrics import accuracy_score

# Performans ölçüm fonksiyonu
def compute_metrics(pred):
    labels = pred.label_ids  # Gerçek etiketler
    preds = pred.predictions.argmax(-1)  # Tahmin edilen etiketler
    acc = accuracy_score(labels, preds)  # Doğruluk oranını hesapla
    return {"accuracy": acc}

print("Performans ölçüm fonksiyonu tanımlandı.")


Performans ölçüm fonksiyonu tanımlandı.


In [ ]:
from transformers import Trainer

# Trainer nesnesini oluştur
trainer = Trainer(
    model=model,                           # Eğitim yapılacak model
    args=training_args,                    # Eğitim ayarları
    train_dataset=tokenized_train,         # Eğitim veri seti
    eval_dataset=tokenized_eval,           # Değerlendirme veri seti
    compute_metrics=compute_metrics        # Performans ölçüm fonksiyonu
)

# Modeli eğit
print("Eğitim başlıyor...")
trainer.train()

print("Eğitim tamamlandı.")


Eğitim başlıyor...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.335600,0.259394,0.904000
2,0.175800,0.346644,0.893000
3,0.071400,0.493404,0.892000
4,0.031900,0.564374,0.891000
5,0.001500,0.593482,0.895000


Eğitim tamamlandı.


In [ ]:
# Test cümleleri
test_texts = [
    # Olumlu yorumlar
    "This movie was absolutely amazing, everyone should watch it.",
    "The acting was brilliant, and the visuals were stunning.",
    "An incredible movie with great acting and visuals.",
    "A masterpiece that touched my heart, simply phenomenal.",
    "One of the best films I have ever seen, highly recommend it.",

    # Olumsuz yorumlar
    "The plot was weak, and the pacing was terrible.",
    "The acting was so bad, it made the movie unwatchable.",
    "The worst movie I have ever seen, a total disaster.",
    "I regret wasting my time on this awful film.",
    "The story was dull, and the characters were poorly developed.",

    # Tarafsız ve karmaşık yorumlar (edge cases)
    "It was okay, not too bad but not amazing either.",
    "Some parts were enjoyable, but others were really boring.",
    "Mixed feelings about this one, it had good and bad moments.",
    "The beginning was great, but the ending ruined the whole experience.",
    "It had potential, but it failed to deliver a compelling story.",

    # Güçlü olumlu ve olumsuz ifadeler
    "An absolute masterpiece! Everything about this movie was perfect.",
    "Completely unwatchable, the worst experience I've had in years.",
    "A cinematic triumph that redefines the genre.",
    "Utterly disappointing, I can't believe this was even made.",
    "The visuals and story were breathtaking, a must-watch for everyone!"
]

# Model üzerinde tahmin
for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    print(f"Text: {text} -> Prediction: {'Positive' if prediction == 1 else 'Negative'}")


Text: This movie was absolutely amazing, everyone should watch it. -> Prediction: Positive
Text: The acting was brilliant, and the visuals were stunning. -> Prediction: Positive
Text: An incredible movie with great acting and visuals. -> Prediction: Positive
Text: A masterpiece that touched my heart, simply phenomenal. -> Prediction: Positive
Text: One of the best films I have ever seen, highly recommend it. -> Prediction: Positive
Text: The plot was weak, and the pacing was terrible. -> Prediction: Negative
Text: The acting was so bad, it made the movie unwatchable. -> Prediction: Negative
Text: The worst movie I have ever seen, a total disaster. -> Prediction: Negative
Text: I regret wasting my time on this awful film. -> Prediction: Negative
Text: The story was dull, and the characters were poorly developed. -> Prediction: Negative
Text: It was okay, not too bad but not amazing either. -> Prediction: Negative
Text: Some parts were enjoyable, but others were really boring. -> Predict

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Model ve Tokenizer'ı yükle
model_path = "/content/results/checkpoint-1565"  # En son checkpoint dizini

# Tokenizer'ı 'distilbert-base-uncased' den yükle
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Modeli checkpoint dizininden yükle
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Modeli uygun cihaza taşı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model ve tokenizer başarıyla yüklendi.")

Model ve tokenizer başarıyla yüklendi.


In [ ]:
def classify_comment(comment):
    # Yorumu tokenize et ve cihaza taşı
    inputs = tokenizer(comment, return_tensors="pt", truncation=True, padding=True).to(device)
    # Modelden tahmin al
    outputs = model(**inputs)
    # Tahmini sınıfı belirle
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    # Sınıfı "positive" ya da "negative" olarak döndür
    return "positive" if prediction == 1 else "negative"

# Örnek bir yorum üzerinde test edelim
test_comment = "Bu film harikaydı, kesinlikle izlemelisiniz!"
print(f"Yorum: {test_comment} -> Tahmin: {classify_comment(test_comment)}")


Yorum: Bu film harikaydı, kesinlikle izlemelisiniz! -> Tahmin: positive


In [ ]:
import json

# Örnek film veri tabanı
film_data = {
    "1": {"title": "Movie A", "score": 100, "reviews": 20},
    "2": {"title": "Movie B", "score": 85, "reviews": 15},
    "3": {"title": "Movie C", "score": 90, "reviews": 18}
}

# Film verilerini kaydetmek için bir dosya oluştur
with open("film_data.json", "w") as file:
    json.dump(film_data, file, indent=4)

def update_film_score(film_id, comment):
    # Film verilerini yükle
    with open("film_data.json", "r") as file:
        film_data = json.load(file)

    if film_id not in film_data:
        return f"Film ID {film_id} not found!"

    # Yorumu sınıflandır
    sentiment = classify_comment(comment)
    print(f"Comment: {comment} -> Sentiment: {sentiment}")

    # Puanı güncelle
    if sentiment == "positive":
        film_data[film_id]["score"] += 5
    else:
        film_data[film_id]["score"] -= 5

    # Toplam yorum sayısını artır
    film_data[film_id]["reviews"] += 1

    # Güncellenmiş veriyi kaydet
    with open("film_data.json", "w") as file:
        json.dump(film_data, file, indent=4)

    return f"{film_data[film_id]['title']} updated! New Score: {film_data[film_id]['score']}, Total Reviews: {film_data[film_id]['reviews']}"


In [ ]:
# Test: Bir film için yorum yap ve puanı güncelle
film_id = "1"  # Örnek: 10 numaralı film
comment = "This movie was absolutely amazing! A must-watch!"  # Örnek olumlu yorum

# Film puanını güncelle
result = update_film_score(film_id, comment)
print(result)



Comment: This movie was absolutely amazing! A must-watch! -> Sentiment: positive
The Shawshank Redemption updated! New Score: 105, Total Reviews: 2500003
